In [1]:
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
data_dir = 'D:/YOGA/dataset'
processed_data_dir = 'D:/YOGA/processed_dataset'

os.makedirs(processed_data_dir, exist_ok=True)

for folder_name in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder_name)

    if os.path.isdir(folder_path):
        processed_folder_path = os.path.join(processed_data_dir, folder_name)
        os.makedirs(processed_folder_path, exist_ok=True)

        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)

            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img = Image.open(img_path).convert('RGB')
                img.save(os.path.join(processed_folder_path, img_name))


In [19]:
img_height, img_width = 180, 180
batch_size = 32

In [20]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [21]:
train_gen = train_datagen.flow_from_directory(
    processed_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    processed_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 1894 images belonging to 33 classes.
Found 454 images belonging to 33 classes.


In [22]:
model = models.Sequential([
    layers.Input(shape=(img_height, img_width, 3)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(len(train_gen.class_indices), activation='softmax')
])

In [23]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [24]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [25]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[early_stopping]
)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 167s 3s/step - accuracy: 0.0390 - loss: 4.4398 - val_accuracy: 0.0330 - val_loss: 4.3405
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.0730 - loss: 3.8827 - val_accuracy: 0.0308 - val_loss: 4.3127
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.0908 - loss: 3.5858 - val_accuracy: 0.0352 - val_loss: 4.1502
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.0982 - loss: 3.4717 - val_accuracy: 0.0507 - val_loss: 4.1474
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.1481 - loss: 3.2705 - val_accuracy: 0.0419 - val_loss: 3.8285
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 157s 3s/step - accuracy: 0.1385 - loss: 3.3131 - val_accuracy: 0.0859 - val_loss: 3.4216
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 157s 3s/step - accuracy: 0.1555 - loss: 3.1491 - val_accuracy: 0.0969 - val_loss: 3.2221
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 173s 3s/step - accuracy: 0.1761 - loss: 2.9836 - val_accuracy: 0.1740 - v

In [ ]:
import cv2
import numpy as np

model.save("yoga_pose_model.keras")  
model = tf.keras.models.load_model("yoga_pose_model.keras")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img = cv2.resize(frame, (img_height, img_width))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0) 

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    class_name = list(train_gen.class_indices.keys())[predicted_class]

    cv2.putText(frame, class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Yoga Pose Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 871ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━